In [85]:
import pandas as pd
import numpy as np
import csv

In [119]:
url = 'https://www.justice.gov/eoir/eoir-immigration-court-listing#MP'
tables = pd.read_html(url)

for i in tables:
    i.columns = i.iloc[0].values
    i.drop(0, inplace=True)
    i.reset_index(drop=True)

df = tables[0]

states = df.columns[0].replace('  ',' ').replace('|', '').split()

for i in range(len(states)):
    if states[i].startswith('New') or states[i].startswith('North') \
    or states[i].startswith('South') or states[i].startswith('Puerto'):
        states[i] = states[i] + ' ' + states[i+1]
        states[i+1] = ''
    if states[i].startswith('Northern'):
        states[i] = states[i] + ' ' + states[i+2]
        states[i+2] = ''
while '' in states:
    states.remove('')

tables[0] = states

judges = tables[1:]

for s, j in zip(states, judges):
    j['State'] = s

df = pd.concat(tables[1:], ignore_index=True)

df.rename(columns = {'Court Administrator':'court_admin',
                     'Immigration Judges':'judges',
                     'Address':'court_address',
                     'Court':'court_name', 
                     'State':'court_state'}, inplace=True)

df.dropna(axis=1, inplace=True)

for i in range(len(df)):
    s = df.court_state[i] 
    if s in cir_dict:
        df['court_circuit'] = cir_dict[s]

def jnames(names):
    return names.split('  ')

df['judges'] = df['judges'].apply(jnames)

df.to_csv('judges_courts.csv', index=False)

In [ ]:
url_laws_acts = 'https://en.wikipedia.org/wiki/List_of_United_States_immigration_laws'
laws_acts = pd.read_html(url_laws_acts)

def cleanit(string):
    if 'Act ' in string:
        s = string[:string.find(' Act ')].strip()
        return s + ' Act'
    
    if 'Law ' in string:
        s = string[:string.find(' Law ')].strip()
        return s + ' Law'

    return string

laws = set(n for n in [cleanit(i) for i in laws_acts[1]['Name of legislation or case'] 
        if type(i) == str] if ' v. ' not in n)

with open('laws_filter.csv', 'w') as f:
    f.write(f'{laws}')

In [ ]:
url_circuits = 'https://en.wikipedia.org/wiki/United_States_courts_of_appeals'
circuits = pd.read_html(url_circuits)

cir_dict = {i:str(j).split(' ')[-2] for i, j in zip(circuits[3]['State'], circuits[3]['Circuit assignment(s)']) \
            if type(j) == str and i != 'District of Columbia'}

for i,j in cir_dict.items():

    if 'st' in j or 'nd' in j or 'rd' in j:
        cir_dict[i] = j[-4:-1]

    elif '0th' in j or '1th' in j:
        cir_dict[i] = j[-5:-1]

    elif 'th' in j:
        cir_dict[i] = j[-4:-1]

cir_dict['District of Columbia'] = 'District of Columbia Circuit'

with open('circuits.csv', 'w') as f:
    f.write(f"{cir_dict}")